## <b><font color='darkblue'>Introduction to A2A</font></b>
([source](https://google.github.io/adk-docs/a2a/intro/)) <font size='3ptx'><b>As you build more complex agentic systems, you will find that a single agent is often not enough. You will want to create specialized agents that can collaborate to solve a problem.</b> The [**Agent2Agent (A2A) Protocol**](https://a2a-protocol.org/) is the standard that allows these agents to communicate with each other.</font>

### <b><font color='darkgreen'>When to Use A2A vs. Local Sub-Agents</font></b>
* <b><font size='3ptx'>Local Sub-Agents</font></b>: These are agents that run within the same application process as your main agent. They are like internal modules or libraries, used to organize your code into logical, reusable components. Communication between a main agent and its local sub-agents is very fast because it happens directly in memory, without network overhead.
* <b><font size='3ptx'>Remote Agents (A2A)</font></b>: These are independent agents that run as separate services, communicating over a network. A2A defines the standard protocol for this communication.

Consider using A2A when:
* The agent you need to talk to is a <b>separate, standalone service</b> (<font color='brown'>e.g., a specialized financial modeling agent</font>).
* The agent is maintained by a <b>different team or organization</b>.
* You need to connect agents written in <b>different programming languages or agent frameworks</b>.
* You want to enforce a <b>strong, formal contract</b> (<font color='brown'>the A2A protocol</font>) <b>between your system's components</b>.

#### <b>When to Use A2A: Concrete Examples</b>
- <font size='3ptx'><b>Integrating with a Third-Party Service</b></font>: Your main agent needs to get real-time stock prices from an external financial data provider. This provider exposes its data through an A2A-compatible agent.
- <font size='3ptx'><b>Microservices Architecture</b></font>: You have a large system broken down into smaller, independent services (<font color='brown'>e.g., an Order Processing Agent, an Inventory Management Agent, a Shipping Agent</font>). A2A is ideal for these services to communicate with each other across network boundaries.
- <font size='3ptx'><b>Cross-Language Communication</b></font>: Your core business logic is in a Python agent, but you have a legacy system or a specialized component written in Java that you want to integrate as an agent. A2A provides the standardized communication layer.
- <font size='3ptx'><b>Formal API Enforcement</b></font>: You are building a platform where different teams contribute agents, and you need <b>a strict contract for how these agents interact to ensure compatibility and stability</b>.

#### <b>When NOT to Use A2A: Concrete Examples (Prefer Local Sub-Agents)</b>
- <b><font size='3ptx'>Internal Code Organization</font></b>: You are breaking down a complex task within a single agent into smaller, manageable functions or modules (<font color='brown'>e.g., a DataValidator sub-agent that cleans input data before processing</font>). These are best handled as local sub-agents for **performance and simplicity**.
- <b><font size='3ptx'>Performance-Critical Internal Operations</font></b>: A sub-agent is responsible for a high-frequency, **low-latency operation that is tightly coupled with the main agent's execution** (<font color='brown'>e.g., a `RealTimeAnalytics` sub-agent that processes data streams within the same application</font>).
- <b><font size='3ptx'>Shared Memory/Context</font></b>: When sub-agents need **direct access to the main agent's internal state or shared memory for efficiency**, A2A's network overhead and serialization/deserialization would be counterproductive.
- <b><font size='3ptx'>Simple Helper Functions</font></b>: For **small, reusable pieces of logic that don't require independent deployment or complex state management**, a simple function or class within the same agent is more appropriate than a separate A2A agent.

### <b><font color='darkgreen'>The A2A Workflow in ADK: A Simplified View</font></b>
([source](https://google.github.io/adk-docs/a2a/intro/#the-a2a-workflow-in-adk-a-simplified-view)) <b><font size='3ptx'><Agent Development Kit (ADK) simplifies the process of building and connecting agents using the A2A protocol. Here's a straightforward breakdown of how it works:</font></b>

1. <b><font size='3ptx'>Making an Agent Accessible (Exposing)</font></b>: You start with an existing ADK agent that you want other agents to be able to interact with. The ADK provides a simple way to "expose" this agent, turning it into an **A2AServer**. This server acts as a public interface, allowing other agents to send requests to your agent over a network. Think of it like setting up a web server for your agent.
2. <b><font size='3ptx'>Connecting to an Accessible Agent (Consuming)</font></b>: In a separate agent (<font color='brown'>which could be running on the same machine or a different one</font>), you'll use a special ADK component called `RemoteA2aAgent`. This `RemoteA2aAgent` acts as a client that knows how to communicate with the **A2AServer** you exposed earlier. It handles all the complexities of network communication, authentication, and data formatting behind the scenes.

From your perspective as a developer, once you've set up this connection, interacting with the remote agent feels just like interacting with a local tool or function. The ADK abstracts away the network layer, making distributed agent systems as easy to work with as local ones.

### <b><font color='darkgreen'>Visualizing the A2A Workflow</font></b>
<font size='3ptx'><b>To further clarify the A2A workflow, let's look at the "before and after" for both exposing and consuming agents, and then the combined system.</b></font>

#### <b>Exposing an Agent</b>

**Before Exposing**: Your agent code runs as a standalone component, but in this scenario, you want to expose it so that other remote agents can interact with your agent.
```
+-------------------+
| Your Agent Code   |
|   (Standalone)    |
+-------------------+
```

**After Exposing**: Your agent code is integrated with an `A2AServer` (an ADK component), making it accessible over a network to other remote agents.
```
+-----------------+
|   A2A Server    |
| (ADK Component) |<--------+
+-----------------+         |
        |                   |
        v                   |
+-------------------+       |
| Your Agent Code   |       |
| (Now Accessible)  |       |
+-------------------+       |
                            |
                            | (Network Communication)
                            v
+-----------------------------+
|       Remote Agent(s)       |
|    (Can now communicate)    |
+-----------------------------+
```

#### <b>Consuming an Agent</b>

<b>Before Consuming</b>: Your agent (<font color='brown'>referred to as the "Root Agent" in this context</font>) is the application you are developing that needs to interact with a remote agent. Before consuming, it lacks the direct mechanism to do so.
```
+----------------------+         +-------------------------------------------------------------+
|      Root Agent      |         |                        Remote Agent                         |
| (Your existing code) |         | (External Service that you want your Root Agent to talk to) |
+----------------------+         +-------------------------------------------------------------+
```

**After Consuming**: Your Root Agent uses a `RemoteA2aAgent` (<font color='brown'>an ADK component that acts as a client-side proxy for the remote agent</font>) to establish communication with the remote agent.
```
+----------------------+         +-----------------------------------+
|      Root Agent      |         |         RemoteA2aAgent            |
| (Your existing code) |<------->|         (ADK Client Proxy)        |
+----------------------+         |                                   |
                                 |  +-----------------------------+  |
                                 |  |         Remote Agent        |  |
                                 |  |      (External Service)     |  |
                                 |  +-----------------------------+  |
                                 +-----------------------------------+
      (Now talks to remote agent via RemoteA2aAgent)
```

#### <b>Final System (Combined View)</b>

This diagram shows how the consuming and exposing parts connect to form a complete A2A system:
```
Consuming Side:
+----------------------+         +-----------------------------------+
|      Root Agent      |         |         RemoteA2aAgent            |
| (Your existing code) |<------->|         (ADK Client Proxy)        |
+----------------------+         |                                   |
                                 |  +-----------------------------+  |
                                 |  |         Remote Agent        |  |
                                 |  |      (External Service)     |  |
                                 |  +-----------------------------+  |
                                 +-----------------------------------+
                                                 |
                                                 | (Network Communication)
                                                 v
Exposing Side:
                                               +-----------------+
                                               |   A2A Server    |
                                               | (ADK Component) |
                                               +-----------------+
                                                       |
                                                       v
                                               +-------------------+
                                               | Your Agent Code   |
                                               | (Exposed Service) |
                                               +-------------------+
```

### <b><font color='darkgreen'>Concrete Use Case: Customer Service and Product Catalog Agents</font></b>
([source](https://google.github.io/adk-docs/a2a/intro/#concrete-use-case-customer-service-and-product-catalog-agents)) <font size='3ptx'><b>Let's consider a practical example: a `Customer Service Agent` that needs to retrieve product information from a separate `Product Catalog Agent`.</b></font>

#### <b>Before A2A</b>

Initially, your `Customer Service Agent` might not have a direct, standardized way to query the `Product Catalog Agent`, especially if it's a separate service or managed by a different team.
```
+-------------------------+         +--------------------------+
| Customer Service Agent  |         |  Product Catalog Agent   |
| (Needs Product Info)    |         | (Contains Product Data)  |
+-------------------------+         +--------------------------+
      (No direct, standardized communication)
```

#### <b>After A2A</b>
By using the A2A Protocol, the **`Product Catalog Agent`** can expose its functionality as an A2A service. Your **`Customer Service Agent`** can then easily consume this service using ADK's `RemoteA2aAgent`.
```
+-------------------------+         +-----------------------------------+
| Customer Service Agent  |         |         RemoteA2aAgent            |
| (Your Root Agent)       |<------->|         (ADK Client Proxy)        |
+-------------------------+         |                                   |
                                    |  +-----------------------------+  |
                                    |  |     Product Catalog Agent   |  |
                                    |  |      (External Service)     |  |
                                    |  +-----------------------------+  |
                                    +-----------------------------------+
                                                 |
                                                 | (Network Communication)
                                                 v
                                               +-----------------+
                                               |   A2A Server    |
                                               | (ADK Component) |
                                               +-----------------+
                                                       |
                                                       v
                                               +------------------------+
                                               | Product Catalog Agent  |
                                               | (Exposed Service)      |
                                               +------------------------+
```

In this setup, first, the **`Product Catalog Agent`** needs to be exposed via an `A2A Server`. Then, the **`Customer Service Agent`** can simply call methods on the `RemoteA2aAgent` as if it were a tool, and the ADK handles all the underlying communication to the **`Product Catalog Agent`**. **This allows for clear separation of concerns and easy integration of specialized agents**.

## <b><font color='darkblue'>Quickstart: Exposing a remote agent via A2A</font></b>
([source](https://google.github.io/adk-docs/a2a/quickstart-exposing/#quickstart-exposing-a-remote-agent-via-a2a)) <font size='3ptx'>This quickstart covers the most common starting point for any developer: <b>"I have an agent. How do I expose it so that other agents can use my agent via A2A?"</b>. This is crucial for building complex multi-agent systems where different agents need to collaborate and interact.</font>

### <b><font color='darkgreen'>Overview</font></b>
<font size='3ptx'><b>This sample demonstrates how you can easily expose an ADK agent so that it can be then consumed by another agent using the A2A Protocol.</b> There are two main ways to expose an ADK agent via A2A.</font>

* <b><font size='3ptx'>by using the `to_a2a(root_agent)` function</font></b>: use this function if you just want to convert an existing agent to work with A2A, and be able to expose it via a server through [**`uvicorn`**](https://uvicorn.dev/), instead of `adk deploy api_server`. This means that you have tighter control over what you want to expose via `uvicorn` when you want to productionize your agent. Furthermore, the `to_a2a()` function auto-generates an agent card based on your agent code.
* <b><font size='3ptx'>by creating your own agent card (<font color='olive'>agent.json</font>) and hosting it using `adk api_server --a2a`</font></b>: There are two main benefits of using this approach. First, `adk api_server --a2a` works with `adk web`, making it easy to use, debug, and test your agent. Second, with `adk api_server`, you can specify a parent folder with multiple, separate agents. Those agents that have an agent card (<font color='olive'>agent.json</font>), will automatically be usable via A2A by other agents through the same server. However, you will need to create your own agent cards. To create an agent card, you can follow the [**A2A Python tutorial**](https://a2a-protocol.org/latest/tutorials/python/1-introduction/).


This quickstart will focus on `to_a2a()`, as it is the easiest way to expose your agent and will also autogenerate the agent card behind-the-scenes. If you'd like to use the `adk api_server` approach, you can see it being used in the [**A2A Quickstart (Consuming) documentation**](https://google.github.io/adk-docs/a2a/quickstart-consuming/).

```
Before:
                                                ┌────────────────────┐
                                                │ Hello World Agent  │
                                                │  (Python Object)   │
                                                | without agent card │
                                                └────────────────────┘

                                                          │
                                                          │ to_a2a()
                                                          ▼

After:
┌────────────────┐                              ┌───────────────────────────────┐
│   Root Agent   │       A2A Protocol           │ A2A-Exposed Hello World Agent │
│(RemoteA2aAgent)│────────────────────────────▶│      (localhost: 8001)         │
│(localhost:8000)│                              └───────────────────────────────┘
└────────────────┘                              
```

The sample consists of :
* <font size='3ptx'><b>Remote Hello World Agent</b> (<font color='olive'>remote_a2a/hello_world/agent.py</font>)</font>: This is the agent that you want to expose so that other agents can use it via A2A. It is an agent that handles dice rolling and prime number checking. It becomes exposed using the `to_a2a()` function and is served using [**`uvicorn`**](https://uvicorn.dev/).
* <font size='3ptx'><b>Root Agent</b> (<font color='olive'>agent.py</font>)</font>: A simple agent that is just calling the remote Hello World agent.

### <b><font color='darkgreen'>Exposing the Remote Agent with the `to_a2a(root_agent)` function</font></b>
You can take an existing agent built using ADK and make it A2A-compatible by simply wrapping it using the `to_a2a()` function. For example, if you have an agent like the following defined in `root_agent`:
```python
# Your agent code here
root_agent = Agent(
    model='gemini-2.0-flash',
    name='hello_world_agent',

    <...your agent code...>
)
```

Then you can make it A2A-compatible simply by using `to_a2a(root_agent)`:
```python
from google.adk.a2a.utils.agent_to_a2a import to_a2a

# Make your agent A2A-compatible
a2a_app = to_a2a(root_agent, port=8001)
```

The `to_a2a()` function will even auto-generate an agent card in-memory behind-the-scenes by extracting skills, capabilities, and metadata from the ADK agent, so that the well-known agent card is made available when the agent endpoint is served using [**uvicorn**](https://uvicorn.dev/).

You can also provide your own agent card by using the <font color='violet'>agent_card</font> parameter. The value can be an <font color='blue'>**AgentCard**</font> object or a path to an agent card JSON file.

Example with an <font color='blue'>**AgentCard**</font> object:
```python
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from a2a.types import AgentCard

# Define A2A agent card
my_agent_card = AgentCard(
    "name": "file_agent",
    "url": "http://example.com",
    "description": "Test agent from file",
    "version": "1.0.0",
    "capabilities": {},
    "skills": [],
    "defaultInputModes": ["text/plain"],
    "defaultOutputModes": ["text/plain"],
    "supportsAuthenticatedExtendedCard": False,
)
a2a_app = to_a2a(root_agent, port=8001, agent_card=my_agent_card)
```

Example with a path to a JSON file:
```python
from google.adk.a2a.utils.agent_to_a2a import to_a2a

# Load A2A agent card from a file
a2a_app = to_a2a(root_agent, port=8001, agent_card="/path/to/your/agent-card.json")
```

Now let's dive into the sample code.

#### <b>1. Getting the Sample Code</b>

First, make sure you have the necessary dependencies installed:
```shell
$ pip install google-adk[a2a]
```

You can clone and navigate to the `a2a_root` sample here:
```shell
$ git clone https://github.com/google/adk-python.git
```

As you'll see, the folder structure is as follows:
```shell
$ pwd
/tmp/adk-python

$ find . -name a2a_root -type d
./contributing/samples/a2a_root

$ cd contributing/samples
# Under adk-python/contributing/samples
$ tree a2a_root/
a2a_root/
├── agent.py
├── README.md
└── remote_a2a
    └── hello_world
        ├── agent.py
        └── __init__.py

3 directories, 4 files
```

##### <b>Root Agent (<font color='olive'>a2a_root/agent.py</font>)</b>
* <b>root_agent</b>: A `RemoteA2aAgent` that connects to the remote A2A service.
* <b>Agent Card URL</b>: Points to the well-known agent card endpoint on the remote server

##### <b>Remote Hello World Agent (<font color='olive'>a2a_root/remote_a2a/hello_world/agent.py</font>)</b>
* **roll_die(sides: int)**: Function tool for rolling dice with state management
* **check_prime(nums: list[int])**: Async function for prime number checking
* **root_agent**: The main agent with comprehensive instructions
* **a2a_app**: The A2A application created using `to_a2a()` utility

- **`a2a_root/remote_a2a/hello_world/agent.py`**:
```python
def roll_die(sides: int, tool_context: ToolContext) -> int:
  """Roll a die and return the rolled result.

  Args:
    sides: The integer number of sides the die has.
    tool_context: the tool context
  Returns:
    An integer of the result of rolling the die.
  """
  ...

async def check_prime(nums: list[int]) -> str:
  """Check if a given list of numbers are prime.

  Args:
    nums: The list of numbers to check.

  Returns:
    A str indicating which number is prime.
  """
  ...

root_agent = Agent(
    model='gemini-2.0-flash',
    name='hello_world_agent',
    description=(
        'hello world agent that can roll a dice of 8 sides and check prime'
        ' numbers.'
    ),
    instruction="""
      You roll dice and answer questions about the outcome of the dice rolls...
      ...
    """,
    tools=[
        roll_die,
        check_prime,
    ],
    # planner=BuiltInPlanner(
    #     thinking_config=types.ThinkingConfig(
    #         include_thoughts=True,
    #     ),
    # ),
    generate_content_config=types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(  # avoid false alarm about rolling dice.
                category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=types.HarmBlockThreshold.OFF,
            ),
        ]
    ),
)

a2a_app = to_a2a(root_agent, port=8001)
```

#### <b>2. Start the Remote A2A Agent server</b>

You can now start the remote agent server, which will host the `a2a_app` within the hello_world agent:
```shell
$ 
# Ensure current working directory is adk-python/contributing/samples
# Start the remote agent using uvicorn
$ uvicorn a2a_root.remote_a2a.hello_world.agent:a2a_app --host localhost --port 8001
...
INFO:     Started server process [1170690]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8001 (Press CTRL+C to quit)
```

#### <b>3. Check that your remote agent is running</b>

You can check that your agent is up and running by visiting the agent card that was auto-generated earlier as part of your `to_a2a()` function in <font color='olive'>a2a_root/remote_a2a/hello_world/agent.py</font>:
```shell
$ wget -qO- http://localhost:8001/.well-known/agent-card.json | python3 -m json.tool
{
    "capabilities": {},
    "defaultInputModes": [
        "text/plain"
    ],
    "defaultOutputModes": [
        "text/plain"
    ],
    "description": "hello world agent that can roll a dice of 8 sides and check prime numbers.",
    "name": "hello_world_agent",
    "preferredTransport": "JSONRPC",
    "protocolVersion": "0.3.0",
    "skills": [
        {
            "description": "hello world agent that can roll a dice of 8 sides and check prime numbers....",
            "id": "hello_world_agent",
            "name": "model",
            "tags": [
                "llm"
            ]
        },
        {
            "description": "Roll a die and return the rolled result...",
            "id": "hello_world_agent-roll_die",
            "name": "roll_die",
            "tags": [
                "llm",
                "tools"
            ]
        },
        {
            ...
        }
    ],
    "supportsAuthenticatedExtendedCard": false,
    "url": "http://localhost:8001",
    "version": "0.0.1"
}           
```

#### <b>4. Run the Main (Consuming) Agent</b>

Now that your remote agent is running, you can launch the dev UI and select `a2a_root` as your agent:

* **`a2a/a2a_root/agent.py`**:
```python
from google.adk.agents.remote_a2a_agent import AGENT_CARD_WELL_KNOWN_PATH
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent

root_agent = RemoteA2aAgent(
    name="hello_world_agent",
    description=(
        "Helpful assistant that can roll dice and check if numbers are prime."
    ),
    agent_card=f"http://localhost:8001/{AGENT_CARD_WELL_KNOWN_PATH}",
)
```

Launch adk web:
```shell
# In a separate terminal, run the adk web server
$ adk web contributing/samples/
...
+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://127.0.0.1:8000.                         |
+-----------------------------------------------------------------------------+

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

To open the adk web server, go to: http://localhost:8000.

##### <b>Example Interactions</b>
Once both services are running, you can interact with the root agent to see how it calls the remote agent via A2A:

![adk_web](images/a2a_1_adk_web.png)

**Simple Dice Rolling**: This interaction uses a local agent, the Roll Agent:
```
User: Roll a 6-sided die
Bot: I rolled a 4 for you.
```

**Prime Number Checking**: This interaction uses a remote agent via A2A, the Prime Agent:
```
User: Is 7 a prime number?
Bot: Yes, 7 is a prime number.
```

**Combined Operations**: This interaction uses both the local Roll Agent and the remote Prime Agent:
```
User: Roll a 10-sided die and check if it's prime
Bot: I rolled an 8 for you.
Bot: 8 is not a prime number.
```

### <b><font color='darkgreen'>Next Step</font></b>
Now that you have created an agent that's exposing a remote agent via an A2A server, the next step is to learn how to consume it from another agent.

## <b><font color='darkblue'>Quickstart: Consuming a remote agent via A2A</font></b>
([source](https://google.github.io/adk-docs/a2a/quickstart-consuming/)) <font size='3ptx'>This quickstart covers the most common starting point for any developer: "**There is a remote agent, how do I let my ADK agent use it via A2A?**". This is crucial for building complex multi-agent systems where different agents need to collaborate and interact.</font>

### <b><font color='darkgreen'>Overview</font></b>
<font size='3ptx'><b>This sample demonstrates the Agent2Agent (A2A) architecture in the Agent Development Kit (ADK), showcasing how multiple agents can work together to handle complex tasks</b>. The sample implements an agent that can roll dice and check if numbers are prime.</font>

```
┌─────────────────┐    ┌──────────────────┐    ┌────────────────────┐
│   Root Agent    │───▶│   Roll Agent     │    │   Remote Prime     │
│  (Local)        │    │   (Local)        │    │   Agent            │
│                 │    │                  │    │  (localhost:8001)  │
│                 │───▶│                  │◀───│                    │
└─────────────────┘    └──────────────────┘    └────────────────────┘
```

The A2A Basic sample consists of:
* **Root Agent** (`root_agent`): The main orchestrator that delegates tasks to specialized sub-agents
* **Roll Agent** (roll_agent): A local sub-agent that handles dice rolling operations
* **Prime Agent** (prime_agent): A remote A2A agent that checks if numbers are prime, this agent is running on a separate A2A server

### <b><font color='darkgreen'>Exposing Your Agent with the ADK Server</font></b>
<font size='3ptx'>The ADK comes with a built-in CLI command, `adk api_server --a2a` to expose your agent using the A2A protocol.</font>

In the `a2a_basic` example, you will first need to expose the `check_prime_agent` via an A2A server, so that the local root agent can use it.

#### <b>1. Getting the Sample Code</b>

First, make sure you have the necessary dependencies installed:
> pip install google-adk[a2a]

You can clone and navigate to the `a2a_basic` sample here:
> git clone https://github.com/google/adk-python.git

As you'll see, the folder structure is as follows:
```shell
$ pwd
/tmp/adk-python

$ tree contributing/samples/a2a_basic/
contributing/samples/a2a_basic/
├── agent.py
├── __init__.py
├── README.md
└── remote_a2a
    └── check_prime_agent
        ├── agent.json
        ├── agent.py
        └── __init__.py

3 directories, 6 files
```

##### <b>Main Agent (`a2a_basic/agent.py`)</b>
* **roll_die(sides: int)**: Function tool for rolling dice
* **roll_agent**: Local agent specialized in dice rolling
* **prime_agent**: Remote A2A agent configuration
* **root_agent**: Main orchestrator with delegation logic

* **`a2a/a2a_basic/agent.py`**:
```python
import random

from google.adk.agents.llm_agent import Agent
from google.adk.agents.remote_a2a_agent import AGENT_CARD_WELL_KNOWN_PATH
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.tools.example_tool import ExampleTool
from google.genai import types


# --- Roll Die Sub-Agent ---
def roll_die(sides: int) -> int:
  """Roll a die and return the rolled result."""
  return random.randint(1, sides)


roll_agent = Agent(
    name="roll_agent",
    description="Handles rolling dice of different sizes.",
    instruction="""
      You are responsible for rolling dice based on the user's request.
      When asked to roll a die, you must call the roll_die tool with the number of sides as an integer.
    """,
    tools=[roll_die],
    generate_content_config=types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(  # avoid false alarm about rolling dice.
                category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=types.HarmBlockThreshold.OFF,
            ),
        ]
    ),
)

example_tool = ExampleTool([
    {
        "input": {
            "role": "user",
            "parts": [{"text": "Roll a 6-sided die."}],
        },
        "output": [
            {"role": "model", "parts": [{"text": "I rolled a 4 for you."}]}
        ],
    },
    ...
])

prime_agent = RemoteA2aAgent(
    name="prime_agent",
    description="Agent that handles checking if numbers are prime.",
    agent_card=(
        f"http://localhost:8001/a2a/check_prime_agent{AGENT_CARD_WELL_KNOWN_PATH}"
    ),
)

root_agent = Agent(
    model="gemini-2.0-flash",
    name="root_agent",
    instruction="""
      You are a helpful assistant that can roll dice and check if numbers are prime...
    """,
    global_instruction=(
        "You are DicePrimeBot, ready to roll dice and check prime numbers."
    ),
    sub_agents=[roll_agent, prime_agent],
    tools=[example_tool],
    generate_content_config=types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(  # avoid false alarm about rolling dice.
                category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=types.HarmBlockThreshold.OFF,
            ),
        ]
    ),
)
```

##### <b>Remote Prime Agent (`a2a_basic/remote_a2a/check_prime_agent/`)</b>
* **agent.py**: Implementation of the prime checking service
* **agent.json**: Agent card of the A2A agent
* **check_prime(nums: list[int])**: Prime number checking algorithm

```shell
$ tree a2a_basic/remote_a2a/check_prime_agent/
remote_a2a/check_prime_agent/
├── agent.json
├── agent.py
└── __init__.py
```

* **`a2a_basic/remote_a2a/check_prime_agent/agent.py`**:
```python
import random

from google.adk import Agent
from google.adk.tools.tool_context import ToolContext
from google.genai import types


async def check_prime(nums: list[int]) -> str:
  """Check if a given list of numbers are prime.

  Args:
    nums: The list of numbers to check.

  Returns:
    A str indicating which number is prime.
  """
  ...


root_agent = Agent(
    model='gemini-2.0-flash',
    name='check_prime_agent',
    description='check prime agent that can check whether numbers are prime.',
    instruction="""
      You check whether numbers are prime.
      ...
    """,
    tools=[
        check_prime,
    ],
    # planner=BuiltInPlanner(
    #     thinking_config=types.ThinkingConfig(
    #         include_thoughts=True,
    #     ),
    # ),
    generate_content_config=types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(  # avoid false alarm about rolling dice.
                category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=types.HarmBlockThreshold.OFF,
            ),
        ]
    ),
)
```

#### <b>2. Start the Remote Prime Agent server</b>

To show how your ADK agent can consume a remote agent via A2A, you'll first need to start a remote agent server, which will host the prime agent (under `check_prime_agent`).
```shell
$ pwd
/usr/local/google/home/johnkclee/Github/ml_articles/google/agent_development_kit

# Start the remote a2a server that serves the check_prime_agent on port 8001
$ adk api_server --a2a --port 8001 a2a/a2a_basic/remote_a2a
...
INFO:     Started server process [2648753]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
```

To enable debug-level logging, you can add <font color='blue'>--log_level debug</font> to your adk api_server, as in:
```shell
$ adk api_server --a2a --port 8001 a2a/a2a_basic/remote_a2a --log_level debug
```

This will give richer logs for you to inspect when testing your agents.

#### <b>3. Look out for the required agent card (agent-card.json) of the remote agent</b>

<b>A2A Protocol requires that each agent must have an agent card that describes what it does.</b>

If someone else has already built the remote A2A agent that you are looking to consume in your agent, then you should confirm that they have an agent card (<font color='olive'>agent-card.json</font>). In the sample, the `check_prime_agent` already has an agent card provided as `2a_basic/remote_a2a/check_prime_agent/agent.json`:
```shell
$ wget -qO- http://localhost:8001/a2a/check_prime_agent/.well-known/agent-card.json | python3 -m json.tool
{
    "capabilities": {},
    "defaultInputModes": [
        "text/plain"
    ],
    "defaultOutputModes": [
        "application/json"
    ],
    "description": "An agent specialized in checking whether numbers are prime...",
    "name": "check_prime_agent",
    "preferredTransport": "JSONRPC",
    "protocolVersion": "0.3.0",
    "skills": [
        {
            "description": "Check if numbers in a list are prime using efficient mathematical algorithms",
            "id": "prime_checking",
            "name": "Prime Number Checking",
            "tags": [
                "mathematical",
                "computation",
                "prime",
                "numbers"
            ]
        }
    ],
    "url": "http://localhost:8001/a2a/check_prime_agent",
    "version": "1.0.0"
}
```

<b><font color='orange'>Notes:</font></b>
> In ADK, you can use a `to_a2a(root_agent)` wrapper which automatically generates an agent card for you. If you're interested in learning more about how to expose your existing agent so others can use it, then please look at the [A2A Quickstart (Exposing) tutorial](https://google.github.io/adk-docs/a2a/quickstart-exposing/).

#### <b>4. Run the Main (Consuming) Agent</b>

In a separate terminal, run the adk web server:
```shell
$ pwd
/usr/local/google/home/johnkclee/Github/ml_articles/google/agent_development_kit

$ adk web a2a/
...
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

##### <b>How it works</b>
The main agent uses the `RemoteA2aAgent()` function to consume the remote agent (`prime_agent` in our example). As you can see below, `RemoteA2aAgent()` requires the `name`, `description`, and the URL of the `agent_card`.

- **`a2a/a2a_basic/agent.py`**:
```python
<...code truncated...>

from google.adk.agents.remote_a2a_agent import AGENT_CARD_WELL_KNOWN_PATH
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent

prime_agent = RemoteA2aAgent(
    name="prime_agent",
    description="Agent that handles checking if numbers are prime.",
    agent_card=(
        f"http://localhost:8001/a2a/check_prime_agent{AGENT_CARD_WELL_KNOWN_PATH}"
    ),
)

<...code truncated>
```

Then, you can simply use the `RemoteA2aAgent` in your agent. In this case, `prime_agent` is used as one of the <font color='violet'>sub-agents</font> in the `root_agent` below:
```python
from google.adk.agents.llm_agent import Agent
from google.genai import types

root_agent = Agent(
    model="gemini-2.0-flash",
    name="root_agent",
    instruction="""
      You are a helpful assistant that can roll dice and check if numbers are prime.
      ...
    """,
    global_instruction=(
        "You are DicePrimeBot, ready to roll dice and check prime numbers."
    ),
    sub_agents=[roll_agent, prime_agent],
    tools=[example_tool],
    generate_content_config=types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(  # avoid false alarm about rolling dice.
                category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=types.HarmBlockThreshold.OFF,
            ),
        ]
    ),
)
```

#### <b>Example Interactions</b>
Once both your main and remote agents are running, you can interact with the root agent to see how it calls the remote agent via A2A:

**Simple Dice Rolling**: This interaction uses a local agent, the Roll Agent:
```
User: Roll a 6-sided die
Bot: I rolled a 4 for you.
```

**Prime Number Checking**: This interaction uses a remote agent via A2A, the Prime Agent:
```
User: Is 7 a prime number?
Bot: Yes, 7 is a prime number.
```

**Combined Operations**: This interaction uses both the local Roll Agent and the remote Prime Agent:
```
User: Roll a 10-sided die and check if it's prime
Bot: I rolled an 8 for you.
Bot: 8 is not a prime number.
```